In [1]:
%matplotlib inline

import cv2
import itertools as it
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import random
from collections import Counter
from utilities import *

def show_square(images, size=None):
    n = math.ceil(math.sqrt(len(images)))
    if size is None:
        size = 999 // n
    height, width = (min(size, n) for n in images[0].shape[:2])
    g = (cv2.resize(image, (width, height)) for image in images)
    z = np.zeros_like(images[0])[:height, :width]
    g = (np.hstack(list(g)) for g in grouper(g, n, fillvalue=z))
    return show_and_wait(np.vstack(list(g)))

In [2]:
g = (fr'D:\Valorant\Killed\Color\{i:05}.pickle' for i in (1, 39))
g = map(pickle_iter, g)
images = list(it.chain.from_iterable(g))
len(images)

16000

In [3]:
for i, image in enumerate(images):
    if show_and_wait(image, wait_time=1) == 'q':
        print(i)
        break
cv2.destroyAllWindows()

1883


In [4]:
indexes = (0, 6096, 6310, 7748, 8000, 11700, 13565, 13645, 13734, 13783,
           13849, 15033, 15545, 15548, 15555, 15582, 15602, 16000)

In [5]:
positives = list(images[a:b] for a, b in grouper(indexes[1:-1], 2))
show_square(list(it.chain.from_iterable(positives)))
cv2.destroyAllWindows()
[(len(l), list(prime_factors(len(l)))) for l in positives]

[(214, [2, 107]),
 (252, [2, 2, 3, 3, 7]),
 (1865, [5, 373]),
 (89, [89]),
 (66, [2, 3, 11]),
 (512, [2, 2, 2, 2, 2, 2, 2, 2, 2]),
 (7, [7]),
 (20, [2, 2, 5])]

In [6]:
show_and_wait(np.hstack([np.vstack(random.sample(l, 7)) for l in positives]))
cv2.destroyAllWindows()

In [7]:
for i, image in enumerate(it.chain.from_iterable(positives)):
    if show_and_wait(image, wait_time=12) == 'q':
        break
cv2.destroyAllWindows()

In [8]:
left, right = 3, 59
width, height = right - left, 11
extents = (
    (59, left, 59 + height, right),
    (95, left, 95 + height, right),
    (49, left, 49 + height, right),
    (73, left, 73 + height, right),
    (73, left, 73 + height, right),
    (60, left, 60 + height, right),
    (60, left, 60 + height, right),
    (60, left, 60 + height, right),
)
g = zip(positives, extents)
g = (zip(a, it.repeat(b)) for a, b in g)
g = it.chain.from_iterable(g)
g = (i[top:bottom, left:right] for i, (top, left, bottom, right) in g)
g = ((i, cv2.cvtColor(i, cv2.COLOR_BGR2LUV)) for i in g)
g = zip(*g)
color = np.stack(list(next(g))).mean(axis=(0, 1, 2))
luv = np.stack(list(next(g))).mean(axis=(0, 1, 2))
show_and_wait(np.vstack([
    np.broadcast_to(color, (111, 222, 3)),
    cv2.cvtColor(np.broadcast_to(luv, (111, 222, 3)), cv2.COLOR_LUV2BGR),
]))
cv2.destroyAllWindows()
positives = list(it.chain.from_iterable(positives))
color, luv, cv2.cvtColor(np.broadcast_to(luv, (1, 1, 3)), cv2.COLOR_LUV2BGR)

(array([0.52949995, 0.6719118 , 0.70135915], dtype=float32),
 array([70.05418  ,  6.4813266, 27.519352 ], dtype=float32),
 array([[[0.5308992, 0.6740422, 0.7026945]]], dtype=float32))

In [9]:
color_template, luv_template = (np.broadcast_to(i, (height, width, 3)) for i in (color, luv))
match = cv2.matchTemplate(positives[0][:, left:right], color_template, cv2.TM_SQDIFF)
show_and_wait(np.broadcast_to(match / match.max(), (match.shape[0], 222)))
cv2.destroyAllWindows()
match.reshape(-1), match.min(), match.max()

(array([ 99.53078 ,  98.58604 ,  97.52049 ,  95.9792  ,  94.303734,
         92.19998 ,  89.75486 ,  86.91403 ,  83.807304,  80.45618 ,
         76.857216,  73.23069 ,  69.96427 ,  66.92392 ,  64.334305,
         61.748333,  59.354908,  57.240044,  55.48504 ,  53.973682,
         52.757607,  51.799583,  50.912483,  49.92861 ,  49.091946,
         48.172413,  47.31106 ,  46.335487,  45.453068,  44.512863,
         43.430042,  42.12786 ,  40.60775 ,  39.19978 ,  37.955402,
         36.667732,  35.610847,  34.65393 ,  33.828552,  32.963394,
         32.222588,  31.631948,  31.19197 ,  30.875448,  30.822918,
         30.747808,  30.097124,  28.216944,  27.151716,  26.13875 ,
         25.076878,  24.038067,  23.013193,  22.003084,  21.17602 ,
         20.083853,  18.978073,  18.5195  ,  19.288967,  19.259623,
         19.875525,  19.502615,  34.749435,  50.551704,  66.93586 ,
         82.95444 ,  97.44619 , 111.53619 , 126.60974 , 140.76355 ,
        153.68468 , 174.26604 , 192.172   , 194.

In [10]:
negatives = list(it.chain.from_iterable(range(a, b) for a, b in grouper(indexes, 2)))
negatives = sorted(random.sample(negatives, len(positives)))
negatives = [images[i] for i in negatives]
show_square(negatives)
cv2.destroyAllWindows()
list(prime_factors(len(negatives))), len(positives), len(negatives)

([5, 5, 11, 11], 3025, 3025)

In [11]:
def fn(image):
    match = cv2.matchTemplate(image[:, left:right], color_template, cv2.TM_SQDIFF)
    return match
image = np.hstack(list(map(fn, negatives + positives)))
show_and_wait(image / image.max())
cv2.destroyAllWindows()

In [12]:
n, d = 25, math.log(2)

In [13]:
image = np.random.rand(9).reshape((3, 3))
image, image[image < .3]

(array([[0.01037415, 0.50187459, 0.49577329],
        [0.13382953, 0.14211109, 0.21855868],
        [0.41850818, 0.24810117, 0.08405965]]),
 array([0.01037415, 0.13382953, 0.14211109, 0.21855868, 0.24810117,
        0.08405965]))

In [14]:
def fn(image):
    image = cv2.cvtColor(image[:, left:right], cv2.COLOR_BGR2LUV)
    match = cv2.matchTemplate(image, luv_template, cv2.TM_SQDIFF)
    result = match.min() < 500000 or len(match[match < 2 * match.min()]) < 60
    return result
    #print(match.min(), match.max(), round(math.log(match.min(), 2)),
     #     round(math.log(match.max(), 2)),
      #    len(match[match < 2 * match.min()]), result)
[Counter(map(fn, images)) for images in (negatives, positives)]

[Counter({False: 2172, True: 853}), Counter({True: 3020, False: 5})]

In [15]:
def fn(image):
    match = cv2.matchTemplate(image[:, left:right], color_template, cv2.TM_SQDIFF)
    result = match.min() < 42 or len(match[match < 2 * match.min()]) < 30
    return result
[Counter(map(fn, images)) for images in (negatives, positives)]

[Counter({False: 2032, True: 993}), Counter({True: 3019, False: 6})]